# PyTorch 中的循环神经网络模块
前面我们讲了循环神经网络的基础知识和网络结构，下面我们教大家如何在 pytorch 下构建循环神经网络，因为 pytorch 的动态图机制，使得循环神经网络非常方便。

## 一般的 RNN

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9xz889xj30kb07nglo.jpg)

对于最简单的 RNN，我们可以使用下面两种方式去调用，分别是 `torch.nn.RNNCell()` 和 `torch.nn.RNN()`，这两种方式的区别在于 `RNNCell()` 只能接受序列中单步的输入，且必须传入隐藏状态，而 `RNN()` 可以接受一个序列的输入，默认会传入全 0 的隐藏状态，也可以自己申明隐藏状态传入。

`RNN()` 里面的参数有

input_size 表示输入 $x_t$ 的特征维度

hidden_size 表示输出的特征维度

num_layers 表示网络的层数

nonlinearity 表示选用的非线性激活函数，默认是 'tanh'

bias 表示是否使用偏置，默认使用

batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位

dropout 表示是否在输出层应用 dropout

bidirectional 表示是否使用双向的 rnn，默认是 False

对于 `RNNCell()`，里面的参数就少很多，只有 input_size，hidden_size，bias 以及 nonlinearity

In [1]:
import torch
from torch.autograd import Variable
from torch import nn

In [2]:
# 定义一个单步的 rnn
rnn_single = nn.RNNCell(input_size=100, hidden_size=200)

In [3]:
# 访问其中的参数
rnn_single.weight_hh

Parameter containing:
tensor([[ 0.0610, -0.0631,  0.0682,  ..., -0.0304,  0.0201, -0.0586],
        [ 0.0426, -0.0698,  0.0014,  ...,  0.0600,  0.0111, -0.0057],
        [ 0.0569, -0.0128, -0.0636,  ...,  0.0062, -0.0413,  0.0564],
        ...,
        [-0.0120,  0.0192, -0.0190,  ...,  0.0328, -0.0200,  0.0470],
        [ 0.0177, -0.0078,  0.0631,  ...,  0.0490, -0.0219,  0.0340],
        [-0.0119, -0.0158,  0.0564,  ..., -0.0453,  0.0399, -0.0006]],
       requires_grad=True)

In [4]:
# 构造一个序列，长为 6，batch 是 5， 特征是 100
x = Variable(torch.randn(6, 5, 100)) # 这是 rnn 的输入格式

In [5]:
# 定义初始的记忆状态
h_t = Variable(torch.zeros(5, 200))

In [6]:
# 传入 rnn
out = []
for i in range(6): # 通过循环 6 次作用在整个序列上
    h_t = rnn_single(x[i], h_t)
    out.append(h_t)

In [7]:
h_t

tensor([[-1.6748e-01,  4.0187e-01,  5.6576e-01, -5.0911e-01,  9.6427e-02,
         -1.1981e-01,  1.0346e-01, -4.1212e-01,  5.1229e-01,  5.6375e-02,
          2.8492e-01, -4.6897e-02, -3.0272e-01, -7.6582e-03,  7.0893e-02,
         -7.6915e-02, -2.0129e-01,  1.7588e-01, -4.2664e-01, -3.3444e-01,
         -4.2900e-01,  4.7092e-01, -2.7584e-01,  1.6589e-01,  3.3437e-01,
         -8.5821e-01, -4.4392e-01,  9.1116e-02, -8.1935e-01,  2.4516e-02,
          8.3866e-01, -5.2694e-01, -5.5631e-02,  3.5048e-01, -1.4915e-01,
         -6.7544e-01,  5.2338e-02, -1.5409e-01, -2.1856e-01, -2.2968e-01,
         -2.9446e-01, -4.2050e-01,  4.3188e-01, -2.0166e-01,  8.4647e-03,
          2.8543e-01,  6.3082e-01,  1.1315e-01, -4.0937e-01,  7.8832e-02,
         -9.6864e-02,  8.9421e-02, -8.4914e-02,  5.5909e-01, -5.5617e-01,
         -6.4923e-01,  7.2114e-01,  2.2287e-01, -3.9929e-01,  3.1994e-01,
         -1.4658e-01,  1.2258e-01, -1.2450e-01,  3.4108e-01, -3.6666e-02,
          2.5762e-01, -3.1889e-02,  8.

In [8]:
len(out)

6

In [9]:
out[0].shape # 每个输出的维度

torch.Size([5, 200])

可以看到经过了 rnn 之后，隐藏状态的值已经被改变了，因为网络记忆了序列中的信息，同时输出 6 个结果

下面我们看看直接使用 `RNN` 的情况

In [10]:
rnn_seq = nn.RNN(100, 200)

In [11]:
# 访问其中的参数
rnn_seq.weight_hh_l0

Parameter containing:
tensor([[ 0.0669, -0.0434,  0.0313,  ..., -0.0551, -0.0300, -0.0013],
        [ 0.0141, -0.0143, -0.0112,  ...,  0.0189, -0.0158,  0.0056],
        [ 0.0199,  0.0174,  0.0429,  ...,  0.0010, -0.0047, -0.0209],
        ...,
        [-0.0149,  0.0423, -0.0652,  ...,  0.0497, -0.0398,  0.0392],
        [ 0.0021, -0.0324, -0.0002,  ...,  0.0094,  0.0113, -0.0195],
        [ 0.0285, -0.0691,  0.0647,  ..., -0.0113,  0.0293,  0.0084]],
       requires_grad=True)

In [12]:
out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [13]:
h_t

tensor([[[-3.5024e-01, -2.0231e-01, -3.3821e-01, -1.2239e-01,  5.0185e-02,
          -6.9162e-02,  7.3102e-01,  2.4910e-01,  4.3213e-01, -2.9700e-01,
          -9.5574e-02,  4.0727e-01, -2.0550e-01,  7.1633e-01,  3.9272e-01,
          -3.6991e-01,  3.5668e-01,  3.0509e-01, -4.7418e-01,  2.1150e-01,
           4.4862e-01, -5.3869e-01, -2.4930e-02,  2.8848e-02, -1.3238e-01,
          -3.5643e-01, -3.1983e-01, -7.9620e-02, -1.8883e-01, -5.1758e-01,
           4.7296e-01,  2.7569e-01,  3.9482e-01, -3.1368e-01,  4.8255e-01,
          -6.0985e-01, -2.9448e-01, -8.5975e-01,  1.3601e-01, -1.2140e-01,
           5.2427e-01,  4.7032e-01, -6.5149e-02, -4.7235e-02, -3.9682e-02,
           1.6384e-01,  1.2841e-01,  5.9862e-01, -4.3275e-01, -2.1130e-01,
           3.1593e-01, -1.4612e-01,  3.5893e-01,  5.1617e-01,  4.9837e-01,
          -1.2484e-01,  1.7031e-01,  3.7729e-02,  3.9202e-01,  4.6480e-01,
          -7.1471e-01, -6.6513e-02, -3.6928e-01, -3.0572e-01, -1.1032e-01,
           5.8368e-02, -2

In [14]:
len(out)

6

这里的 h_t 是网络最后的隐藏状态，网络也输出了 6 个结果

In [15]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(1, 5, 200))

这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)

In [16]:
out, h_t = rnn_seq(x, h_0)

In [17]:
h_t

tensor([[[-3.4959e-01, -2.0528e-01, -3.3781e-01, -1.1640e-01,  5.9273e-02,
          -9.4274e-02,  7.2740e-01,  2.6074e-01,  4.5294e-01, -2.9178e-01,
          -1.0350e-01,  4.0115e-01, -2.0031e-01,  7.2366e-01,  3.9836e-01,
          -3.6514e-01,  3.5592e-01,  3.1905e-01, -4.8032e-01,  2.0054e-01,
           4.4729e-01, -5.4620e-01, -3.0396e-02,  6.1709e-02, -1.0244e-01,
          -3.6023e-01, -3.1668e-01, -6.5274e-02, -1.8378e-01, -5.1996e-01,
           4.8063e-01,  2.7307e-01,  4.0960e-01, -3.0244e-01,  4.7116e-01,
          -6.0066e-01, -2.8026e-01, -8.6161e-01,  1.3177e-01, -1.1110e-01,
           5.2717e-01,  4.5789e-01, -6.9555e-02, -4.7753e-02, -2.6491e-02,
           1.6731e-01,  1.4521e-01,  6.1027e-01, -4.1724e-01, -2.1739e-01,
           3.1764e-01, -1.4183e-01,  3.5431e-01,  5.1288e-01,  5.0238e-01,
          -1.2372e-01,  1.7121e-01,  9.7708e-03,  3.8551e-01,  4.7979e-01,
          -7.0781e-01, -5.6723e-02, -3.6418e-01, -2.9352e-01, -9.1929e-02,
           2.3524e-02, -2

In [18]:
out.shape

torch.Size([6, 5, 200])

同时输出的结果也是 (seq, batch, feature)

一般情况下我们都是用 `nn.RNN()` 而不是 `nn.RNNCell()`，因为 `nn.RNN()` 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态

## LSTM

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9qj3uhmj30iz07ct90.jpg)

LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 `nn.LSTMCell()` 和 `nn.LSTM()` 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子

In [19]:
lstm_seq = nn.LSTM(50, 100, num_layers=2) # 输入维度 100，输出 200，两层

In [20]:
lstm_seq.weight_hh_l0 # 第一层的 h_t 权重

Parameter containing:
tensor([[-0.0474,  0.0675, -0.0849,  ...,  0.0972,  0.0453,  0.0122],
        [ 0.0347, -0.0532,  0.0234,  ...,  0.0745, -0.0058,  0.0825],
        [-0.0502, -0.0521,  0.0204,  ...,  0.0876,  0.0988,  0.0520],
        ...,
        [-0.0762, -0.0352,  0.0391,  ..., -0.0654,  0.0306,  0.0976],
        [ 0.0942, -0.0459,  0.0304,  ..., -0.0129,  0.0483, -0.0801],
        [-0.0962, -0.0277,  0.0238,  ..., -0.0139, -0.0364, -0.0167]],
       requires_grad=True)

**小练习：想想为什么这个系数的大小是 (400, 100)**

In [21]:
lstm_input = Variable(torch.randn(10, 3, 50)) # 序列 10，batch 是 3，输入维度 50

In [22]:
out, (h, c) = lstm_seq(lstm_input) # 使用默认的全 0 隐藏状态

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [23]:
h.shape # 两层，Batch 是 3，特征是 100

torch.Size([2, 3, 100])

In [24]:
c.shape

torch.Size([2, 3, 100])

In [25]:
out.shape

torch.Size([10, 3, 100])

我们可以不使用默认的隐藏状态，这是需要传入两个张量

In [26]:
h_init = Variable(torch.randn(2, 3, 100))
c_init = Variable(torch.randn(2, 3, 100))

In [27]:
out, (h, c) = lstm_seq(lstm_input, (h_init, c_init))

In [28]:
h.shape

torch.Size([2, 3, 100])

In [29]:
c.shape

torch.Size([2, 3, 100])

In [30]:
out.shape

torch.Size([10, 3, 100])

# GRU
![](https://ws3.sinaimg.cn/large/006tKfTcly1fmtaj38y9sj30io06bmxc.jpg)

GRU 和前面讲的这两个是同样的道理，就不再细说，还是演示一下例子

In [31]:
gru_seq = nn.GRU(10, 20)
gru_input = Variable(torch.randn(3, 32, 10))

out, h = gru_seq(gru_input)

In [32]:
gru_seq.weight_hh_l0

Parameter containing:
tensor([[ 0.1402, -0.0994,  0.0085,  ...,  0.1306, -0.0543, -0.1815],
        [-0.0601, -0.1165,  0.2182,  ...,  0.1575,  0.1400, -0.0903],
        [ 0.1173, -0.1882,  0.1372,  ..., -0.0341, -0.0699,  0.1777],
        ...,
        [-0.0797, -0.1880,  0.1477,  ...,  0.2175, -0.0602,  0.0862],
        [ 0.1281,  0.1936, -0.0586,  ..., -0.0213,  0.0916,  0.0288],
        [ 0.0504, -0.0652, -0.0291,  ..., -0.0094,  0.0225, -0.0035]],
       requires_grad=True)

In [33]:
h.shape

torch.Size([1, 32, 20])

In [34]:
out.shape

torch.Size([3, 32, 20])